In [1]:
from chi import server, context, lease
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

username = os.getenv('USER') # all exp resources will have this prefix

In [2]:
l = lease.Lease(f"ML_pipeline-{username}", duration=datetime.timedelta(hours=6))
l.add_flavor_reservation(id=chi.server.get_flavor_id("m1.medium"), amount=1)
l.submit(idempotent=True)

Waiting for lease to start...


Lease ML_pipeline-dswdfb_missouri_edu has reached status active


In [3]:
l.show()

HTML(value='\n        <h2>Lease Details</h2>\n        <table>\n            <tr><th>Name</th><td>ML_pipeline-ds…

Lease Details:
Name: ML_pipeline-dswdfb_missouri_edu
ID: 213b241a-d61f-451e-9c1a-7323b425b4db
Status: ACTIVE
Start Date: 2025-10-28 16:22:00
End Date: 2025-10-28 22:22:00
User ID: 1d5182b969191c3674ff695880eb6212a98ee0f73ce24bb5e8c234ebb25477f7
Project ID: 42c6889d45d74cc8bc6be5a377d21a05

Node Reservations:

Floating IP Reservations:

Network Reservations:

Flavor Reservations:
ID: 1627cb73-b19f-4419-94f0-3efe9230d5ff, Status: active, Flavor: 1627cb73-b19f-4419-94f0-3efe9230d5ff, Amount: 1

Events:


In [4]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-eval-offline-{username}", 
    image_name="CC-Ubuntu24.04",
    flavor_name=l.get_reserved_flavors()[0].name
)
s.submit(idempotent=True)

The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.


Waiting for server node-eval-offline-dswdfb_missouri_edu's status to become ACTIVE. This typically takes 10 minutes for baremetal, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-eval-offline-dswdfb_missouri_edu
Id,e9d6d0f9-da3e-4ba1-a8b9-46d2814a1943
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,reservation:1627cb73-b19f-4419-94f0-3efe9230d5ff
Addresses,sharednet1: IP: 10.56.1.10 (v4) Type: fixed MAC: fa:16:3e:c4:ed:22
Network Name,sharednet1
Created At,2025-10-28T16:23:36Z
Keypair,dswdfb_missouri_edu-jupyter
Reservation Id,None
Host Id,2301d9f41d613700810b3b1a244eabd675bc203843d5aafef904dab6


In [5]:
s.associate_floating_ip()

The python binding code in neutronclient is deprecated in favor of OpenstackSDK, please use that as this will be removed in a future release.


'129.114.25.86'

In [6]:
s.refresh()
s.check_connectivity()

Checking connectivity to 129.114.25.86 port 22.


Connection successful


In [7]:
s.refresh()
s.show(type="widget")

Attribute,node-eval-offline-dswdfb_missouri_edu
Id,e9d6d0f9-da3e-4ba1-a8b9-46d2814a1943
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,reservation:1627cb73-b19f-4419-94f0-3efe9230d5ff
Addresses,sharednet1: IP: 10.56.1.10 (v4) Type: fixed MAC: fa:16:3e:c4:ed:22 IP: 129.114.25.86 (v4) Type: floating MAC: fa:16:3e:c4:ed:22
Network Name,sharednet1
Created At,2025-10-28T16:23:36Z
Keypair,dswdfb_missouri_edu-jupyter
Reservation Id,None
Host Id,2301d9f41d613700810b3b1a244eabd675bc203843d5aafef904dab6


In [8]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

/opt/conda/lib/python3.12/site-packages/paramiko/client.py:885: UserWarning: Unknown ssh-ed25519 host key for 129.114.25.86: b'60dc009e508c16b37f0ee893001d66b6'
  warnings.warn(


# Executing docker install script, commit: 86415efcfe5f8d966625843da41a0f798238cce5


+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install ca-certificates curl >/dev/null
+ sh -c install -m 0755 -d /etc/apt/keyrings
+ sh -c curl -fsSL "https://download.docker.com/linux/ubuntu/gpg" -o /etc/apt/keyrings/docker.asc
+ sh -c chmod a+r /etc/apt/keyrings/docker.asc
+ sh -c echo "deb [arch=amd64 signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu noble stable" > /etc/apt/sources.list.d/docker.list
+ sh -c apt-get -qq update >/dev/null
+ sh -c DEBIAN_FRONTEND=noninteractive apt-get -y -qq install docker-ce docker-ce-cli containerd.io docker-compose-plugin docker-ce-rootless-extras docker-buildx-plugin docker-model-plugin >/dev/null

Running kernel seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.
+ sh -c docker version


Client: Docker Engine - Community
 Version:           28.5.1
 API version:       1.51
 Go version:        go1.24.8
 Git commit:        e180ab8
 Built:             Wed Oct  8 12:17:26 2025
 OS/Arch:           linux/amd64
 Context:           default

Server: Docker Engine - Community
 Engine:
  Version:          28.5.1
  API version:      1.51 (minimum version 1.24)
  Go version:       go1.24.8
  Git commit:       f8215cc
  Built:            Wed Oct  8 12:17:26 2025
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          v1.7.28
  GitCommit:        b98a3aace656320842a23f4a392a33f46af97866
 runc:
  Version:          1.3.0
  GitCommit:        v1.3.0-0-g4ca628d1
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


To run Docker as a non-privileged user, consider setting up the
Docker daemon in rootless mode for your user:

    dockerd-rootless-setuptool.sh install

Visit https://docs.docker.com/go/rootless/ to learn about rootless mode.



<Result cmd='sudo groupadd -f docker; sudo usermod -aG docker $USER' exited=0>

In [ ]:
Create dataa

In [ ]:
mkdir -p eval-offline-chi/docker
mkdir -p eval-offline-chi/scripts


In [ ]:
nano eval-offline-chi/scripts/prepare_qasper.py


In [ ]:
import os
import pandas as pd
from datasets import load_dataset

# Make output folder
os.makedirs("/mnt/Qasper", exist_ok=True)

# Load the QASPER dataset
print("📦 Downloading allenai/qasper dataset ...")
ds = load_dataset("allenai/qasper", split="train")

rows = []
for paper in ds:
    if paper["qas"]["question"]:
        rows.append({
            "id": paper["id"],
            "title": paper["title"],
            "n_questions": len(paper["qas"]["question"])
        })

df = pd.DataFrame(rows)
out_path = "/mnt/Qasper/summary.csv"
df.to_csv(out_path, index=False)

print(f"✅ Saved {out_path} with {len(df)} papers")


In [ ]:
nano eval-offline-chi/docker/docker-compose-data.yaml


In [ ]:
version: "3.8"

services:
  prepare-qasper:
    image: python:3.11-slim
    container_name: prepare-qasper
    volumes:
      - qasperdata:/mnt
      - ../scripts:/scripts
    working_dir: /scripts
    command: >
      bash -c "apt-get update -y &&
      apt-get install -y git &&
      pip install pandas datasets==2.19.1 &&
      python prepare_qasper.py"
    restart: "no"

volumes:
  qasperdata:
    external: true


In [ ]:
docker run --rm -it -v qasperdata:/mnt alpine ls -l /mnt/Qasper


In [ ]:
nano eval-offline-chi/scripts/prepare_hotpot.py


In [ ]:
import os
import json
import pandas as pd
import kagglehub

os.makedirs("/mnt/HotpotQA", exist_ok=True)

print("Downloading HotpotQA dataset via KaggleHub...")

path = kagglehub.dataset_download("jeromeblanchet/hotpotqa-question-answering-dataset")
print(f"Path to HotpotQA dataset: {path}")

json_path = os.path.join(path, "hotpot_dev_distractor_v1.json")
if not os.path.exists(json_path):
    raise FileNotFoundError(f"Could not find {json_path}")

with open(json_path, "r") as f:
    hotpot_data = json.load(f)

print(f"Loaded {len(hotpot_data)} examples")

rows = []
for ex in hotpot_data:
    rows.append({
        "id": ex["_id"],
        "question": ex["question"],
        "answer": ex["answer"],
        "context": ex["context"],
        "supporting_facts": ex["supporting_facts"]
    })

df = pd.DataFrame(rows)
out_csv = "/mnt/HotpotQA/processed_hotpot_df.csv"
df.to_csv(out_csv, index=False)

print(f"Saved processed HotpotQA CSV to {out_csv}")


In [ ]:
version: "3.8"

services:
  prepare-hotpot:
    image: python:3.11-slim
    container_name: prepare-hotpot
    working_dir: /scripts
    volumes:
      - hotpotdata:/mnt
      - ../scripts:/scripts
    command: >
      bash -c "apt-get update -y &&
      apt-get install -y git &&
      pip install --upgrade pip &&
      pip install pandas kagglehub &&
      python prepare_hotpot.py"
    restart: "no"

volumes:
  hotpotdata:
    external: true


In [ ]:
docker volume create hotpotdata


In [ ]:
docker run --rm -it -v hotpotdata:/mnt alpine ls -l /mnt/HotpotQA


In [ ]:
nano eval-offline-chi/docker/docker-compose-datasets.yaml


In [ ]:
version: "3.8"

services:
  prepare-qasper:
    image: python:3.11-slim
    container_name: prepare-qasper
    working_dir: /scripts
    volumes:
      - qasperdata:/mnt
      - ../scripts:/scripts
    command: >
      bash -c "apt-get update -y &&
      apt-get install -y git &&
      pip install --upgrade pip &&
      pip install pandas datasets==2.19.1 &&
      python prepare_qasper.py"
    restart: "no"

  prepare-hotpot:
    image: python:3.11-slim
    container_name: prepare-hotpot
    working_dir: /scripts
    volumes:
      - hotpotdata:/mnt
      - ../scripts:/scripts
    command: >
      bash -c "apt-get update -y &&
      apt-get install -y git &&
      pip install --upgrade pip &&
      pip install pandas kagglehub &&
      python prepare_hotpot.py"
    restart: "no"

volumes:
  qasperdata:
    external: true
  hotpotdata:
    external: true


In [ ]:
docker volume create qasperdata
docker volume create hotpotdata


In [ ]:
docker compose -f eval-offline-chi/docker/docker-compose-datasets.yaml up


In [ ]:
docker run --rm -it -v qasperdata:/mnt alpine ls -l /mnt/Qasper
docker run --rm -it -v hotpotdata:/mnt alpine ls -l /mnt/HotpotQA

In [ ]:
docker run -d --rm \
  -p 8888:8888 \
  -v ~/eval-offline-chi/workspace:/home/jovyan/work/ \
  -v qasperdata:/mnt/Qasper \
  -v hotpotdata:/mnt/HotpotQA \
  -e QASPER_DATA_DIR=/mnt/Qasper \
  -e HOTPOT_DATA_DIR=/mnt/HotpotQA \
  --name jupyter \
  quay.io/jupyter/pytorch-notebook:pytorch-2.5.1


In [ ]:
cc@node-eval-offline-dswdfb-missouri-edu:~/eval-offline-chi$ cd ~/eval-offline-chi
cc@node-eval-offline-dswdfb-missouri-edu:~/eval-offline-chi$ mkdir -p models/{tinyroberta,roberta-base,bert-large}
cc@node-eval-offline-dswdfb-missouri-edu:~/eval-offline-chi$ 

In [ ]:
nano ~/eval-offline-chi/models/server.py


In [ ]:
import os, time, asyncio, grpc, torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import qa_pb2, qa_pb2_grpc

class QAServicer(qa_pb2_grpc.QAServerServicer):
    def __init__(self, model_name):
        self.tok = AutoTokenizer.from_pretrained(model_name)
        self.mdl = AutoModelForQuestionAnswering.from_pretrained(model_name).eval()

    async def Answer(self, request, context):
        t0 = time.perf_counter()
        ins = self.tok(request.question, return_tensors="pt", truncation=True, max_length=384)
        t1 = time.perf_counter()
        with torch.no_grad():
            out = self.mdl(**ins)
        si = int(out.start_logits.argmax()); ei = int(out.end_logits.argmax())
        ans = self.tok.decode(ins.input_ids[0, si:ei+1], skip_special_tokens=True)
        inf_ms = (time.perf_counter()-t1)*1000
        e2e_ms = (time.perf_counter()-t0)*1000
        conf = float(out.start_logits.softmax(-1)[0,si] * out.end_logits.softmax(-1)[0,ei])
        return qa_pb2.AnswerResponse(answer=ans, confidence=conf,
                                     retrieval_ms=0.0, inference_ms=inf_ms, end_to_end_ms=e2e_ms)

async def serve():
    port = int(os.environ.get("PORT","50051"))
    model = os.environ.get("MODEL_NAME","deepset/tinyroberta-squad2")
    server = grpc.aio.server()
    qa_pb2_grpc.add_QAServerServicer_to_server(QAServicer(model), server)
    server.add_insecure_port(f"[::]:{port}")
    await server.start()
    print(f"Serving {model} on :{port}", flush=True)
    await server.wait_for_termination()

if __name__ == "__main__":
    asyncio.run(serve())


In [ ]:
nano qa.proto


In [ ]:
syntax = "proto3";
package qa;

service QAServer {
  rpc Answer(Question) returns (AnswerResponse) {}
}

message Question {
  string question = 1;
  string paper_id = 2;
}

message AnswerResponse {
  string answer = 1;
  double confidence = 2;
  double retrieval_ms = 3;
  double inference_ms = 4;
  double end_to_end_ms = 5;
}


In [ ]:
cd ~/eval-offline-chi
python3 -m venv venv
source venv/bin/activate


In [ ]:
pip install --upgrade pip
pip install grpcio grpcio-tools transformers torch


In [ ]:
python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. qa.proto


In [ ]:
cp qa_pb2.py qa_pb2_grpc.py models/tinyroberta/
cp qa_pb2.py qa_pb2_grpc.py models/roberta-base/
cp qa_pb2.py qa_pb2_grpc.py models/bert-large/


In [ ]:
ls models/tinyroberta


In [ ]:
FROM python:3.11-slim
WORKDIR /app
COPY server.py qa_pb2.py qa_pb2_grpc.py /app/
RUN pip install --no-cache-dir torch transformers grpcio protobuf
ENV MODEL_NAME=deepset/tinyroberta-squad2
ENV PORT=50051
CMD ["python", "server.py"]


In [ ]:
docker build -t qa-tinyroberta models/tinyroberta
docker build -t qa-roberta-base models/roberta-base
docker build -t qa-bert-large models/bert-large


In [ ]:
docker run -d --name tinyroberta-debug \
  -p 50051:50051 \
  -v ~/.cache/huggingface:/root/.cache/huggingface \
  qa-tinyroberta

docker run -d --name roberta-base \
  -p 50052:50051 \
  -v ~/.cache/huggingface:/root/.cache/huggingface \
  qa-roberta-base

docker run -d --name bert-large \
  -p 50053:50051 \
  -v ~/.cache/huggingface:/root/.cache/huggingface \
  qa-bert-large


In [ ]:
cd ~/eval-offline-chi
mkdir -p models/{tinyroberta,roberta-base,bert-large,bert-large-onnx-i8}
Each folder will contain:

A Dockerfile

A server.py (the gRPC or FastAPI inference server)

The model weights (downloaded at build time)

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import grpc, asyncio, time
import qa_pb2, qa_pb2_grpc  # compiled from qa.proto

class QAServicer(qa_pb2_grpc.QAServerServicer):
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForQuestionAnswering.from_pretrained(model_name)
        self.model.eval()

    async def Answer(self, request, context):
        t0 = time.time()
        inputs = self.tokenizer(request.question, return_tensors="pt")
        t1 = time.time()
        with torch.no_grad():
            outputs = self.model(**inputs)
        answer = self.tokenizer.decode(
            inputs["input_ids"][0][torch.argmax(outputs.start_logits): torch.argmax(outputs.end_logits)+1]
        )
        t2 = time.time()
        return qa_pb2.AnswerResponse(
            answer=answer,
            confidence=float(outputs.start_logits.softmax(-1).max()),
            retrieval_ms=0.0,
            inference_ms=(t2-t1)*1000,
            end_to_end_ms=(t2-t0)*1000,
        )

async def serve(model_name, port):
    server = grpc.aio.server()
    qa_pb2_grpc.add_QAServerServicer_to_server(QAServicer(model_name), server)
    server.add_insecure_port(f"[::]:{port}")
    await server.start()
    print(f"{model_name} ready on port {port}")
    await server.wait_for_termination()

if __name__ == "__main__":
    import os
    asyncio.run(serve(
        os.environ.get("MODEL_NAME","deepset/tinyroberta-squad2"),
        int(os.environ.get("PORT","50051"))
    ))


In [ ]:
cp tinyroberta/server.py roberta-base/
cp tinyroberta/server.py bert-large/
cp tinyroberta/server.py bert-large-onnx-i8/


In [ ]:
cd ~/eval-offline-chi
nano qa.proto


In [ ]:
syntax = "proto3";
package qa;

service QAServer {
  rpc Answer(Question) returns (AnswerResponse) {}
}

message Question {
  string question = 1;
  string paper_id = 2; // optional
}

message AnswerResponse {
  string answer = 1;
  double confidence = 2;
  double retrieval_ms = 3;
  double inference_ms = 4;
  double end_to_end_ms = 5;
}
